## 訓練パイプライン

- パイプライン
    - 様々なモジュールを順番に組み合わせて実行すること
    - 特徴洗濯、前処理、ランダムフォレスト、クラスタリングなどなど

- 今回は入力データ点から上位k個の特徴量を洗濯し、ERT分類器を使って分類する

In [6]:
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

# 25次元の特徴ベクトルで表現される150個のラベル付き教師データを生成する
X, y = samples_generator.make_classification(n_samples=150,
                n_features=25, n_classes=3, n_informative=6,
                n_redundant=0, random_state=7)

# 上位k（=9）個の特徴量を選択する
k_best_selector = SelectKBest(f_regression, k=9)

# 推定器が60個で最大深さ4のERT分類器を作る
classifier = ExtraTreesClassifier(n_estimators=60, max_depth=4)

# パイプラインを作成する
# k_best_selectorを'selector'、classifierを'erf'という名前でブロックを作成する
processor_pipeline = Pipeline([('selector', k_best_selector), ('erf', classifier)])

# パイプラインに指定したブロックには引数を渡せる
# ブロック名__引数名
processor_pipeline.set_params(selector__k=7, erf__n_estimators=30)

# 作ったパイプラインで訓練する
processor_pipeline.fit(X, y)

# 入力値Xに対して推論する
output = processor_pipeline.predict(X)
print("Predicted output：\n", output)

# 処理の分類性能
print("\nScore:", processor_pipeline.score(X, y))

Predicted output：
 [0 2 2 0 2 0 2 1 0 1 1 2 1 0 2 2 1 0 0 1 0 2 1 1 2 2 0 0 1 2 1 2 1 0 2 2 1
 1 2 2 2 0 1 2 2 1 1 2 1 0 1 2 2 2 2 0 2 2 0 2 2 0 1 0 2 1 1 1 1 2 0 0 0 2
 0 0 1 2 2 0 0 2 2 2 2 0 0 0 2 2 2 1 2 0 2 0 2 2 0 0 1 1 1 1 2 2 2 2 0 1 1
 0 2 1 1 0 1 1 1 1 0 0 0 1 2 0 0 0 2 1 2 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 2 0
 2 2]

Score: 0.9133333333333333


In [13]:
# 特徴抽出の状態を取得する
status = processor_pipeline.named_steps['selector'].get_support()

# statusは25個
print(len(status))

# Trueが有効と判断された特徴量
print(status)

# 有効な特徴量のインデックス番号を表示するためにごにょごにょする
selected = [i for i, x in enumerate(status) if x]
print("\nIndices of selected features:", ', '.join([str(x) for x in selected]))

25
[False False False False  True False False  True  True False False False
  True False  True False False  True False False False False  True False
 False]

Indices of selected features: 4, 7, 8, 12, 14, 17, 22
